# 온라인 결제 사기거래 탐지

> 목차

- 문제상황 및 데이터 살펴보기
- 문제해결 프로세스 정의
- Data 전처리 및 EDA
- Feature 상관관계 분석
- 이상 탐지 모델링

In [3]:
# Warnings 제거
import warnings
warnings.filterwarnings('ignore')

# 대다수의 이상탐지 : 정상/이상 분류(Classification)
# 라벨링이 있는 슈퍼바이즈드의 경우 트리 모델이 잘 됨

# 문제상황 및 데이터 살펴보기

> 시나리오

- A사는 요즘 핫한 중고거래 플랫폼(사이트)를 운영하는 회사다. 최근 플랫폼 내 페이 시스템을 오픈하고, 페이 시스템을 기반으로 서비스를 확장해 나가고 있다. 중고거래 특성상 빈번하게 사기거래가 발생하게 되는데, 최근 사기거래 건수가 급증하고 있어 소비자 피해가 문제되고 있다. 이를 방지하기 위해서 페이 시스템을 통해 수집된 데이터를 기반으로 이상 거래를 사전 탐지하고 방어 활동들을 전개해나가고자 한다.

> 데이터 살펴보기

- 온라인 거래 데이터


데이터명세
1. step : 1step이 1시간인 시간 단위
2. type : 온라인 거래유형
3. amount : 거래 금액
4. nameOrig : 거래 시작 고객
5. oldbalanceOrg : 거래 전 잔액
6. nameDest : 거래 수신 고객
7. oldbalanceOrg : 거래 전 잔액
8. newbalanceOrig : 거래 후 잔액
9. nameDest : 거래 수신 고객
10. oldbalanceDest : 거래 전 수취인의 초기 잔액
11. newbalanceDest : 거래 전 수취인의 초기 잔액
12. isFraud : 사기 거래

In [6]:
# pd.set_option
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# Data read
df = pd.read_csv("D:\\csv\\chapter05_df.csv")
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


# 문제해결 프로세스 정의

> 문제정의

- 중고 플랫폼 온라인 거래 사기 거래 건수 증가
- 소비자 피해 및 사용자 감소

> 기대효과

- 사기거래 사전 차단으로 소비자 피해 방지
- 신규 회원 증가

> 해결방안 : 사기거래 이상탐지 모델링을 통한 사기거래 사전 차단

- Data전처리 및 EDA
- Feature 상관관계 분석
- 이상 탐지 모델링 수행

> 성과측정 (KPI)

- 모델 활용 전/후 사기거래 건수 비교
- 모델 활용 전/후 사기거래 소비자 피해 금액 비교

> 현업적용

- 온라인 거래 발생 시 거래 데이터 Model Input
- 이상 거래 확인 시 해당 거래 상세 분석 후 조치

> 주요 코드 미리 살펴보기
- 데이터 전처리 및 EDA : list.append(), pd.concat()
- Feature 상관관계 분석 : .value_counts(normalize=True), .groupby('type')['isFraud'].agg(['count', 'sum']), enumerate()
- 모델링 : .drop(), OneHotEncoder(), LogisticRegression()

# 데이터 전처리 및 EDA

### 데이터 전처리
1. Data shape 확인
2. Data Type 확인
3. Null 확인
4. Outlier 확인

In [8]:
# 보유 데이터의 rows와 columns 확인
df.shape

(6362620, 11)

In [9]:
# Data Type 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [10]:
# Null 갯수 확인
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [11]:
# Outlier 확인 (-가 될 수 없는 값이 - 값으로 존재하는지 확인)
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


### 데이터 EDA

- 데이터 특성 파악을 위해서 탐색하는 과정

In [12]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [14]:
# 데이터가 굉장히 불균형함
df['isFraud'].value_counts()

0    6354407
1       8213
Name: isFraud, dtype: int64

In [15]:
# 99.87% : 0.13%
df['isFraud'].value_counts(normalize=True)

0    0.998709
1    0.001291
Name: isFraud, dtype: float64

In [16]:
# 의미가 불분명한 데이터 확인
df['isFlaggedFraud'].value_counts()

0    6362604
1         16
Name: isFlaggedFraud, dtype: int64

In [19]:
# 숫자형, 범주형 변수 분할
numerical_list = []
categorical_list = []

for i in df.columns:
    # Object
    if df[i].dtypes == 'O':
        categorical_list.append(i)
    # int or float or else
    else:
        numerical_list.append(i)
        
print("categorical_list :", categorical_list)
print("numerical_list :", numerical_list)

categorical_list : ['type', 'nameOrig', 'nameDest']
numerical_list : ['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud']


In [20]:
# 범주형 변수에서 '유니크값' 이 너무 많은 경우 (ex - 사용자ID)
# 1. 버린다
# 2. 재그룹핑을 한다 (범주별로 집단화하거나 개인을 지역인 기준으로 등)

# nameOrig : 판매자 아이디 (6353307 가지)
# nameDest : 구매자 아이디 (2722362 가지)
list_of_df = []

for var in categorical_list:
    loop_df = pd.DataFrame({'var':[var], 'ncnt':[df[var].nunique()]})
    list_of_df.append(loop_df)

df_concat = pd.concat(list_of_df).reset_index(drop=True)

# 활용하기 어려울 것 같아 두 변수는 drop
df_concat

,var,ncnt
0,type,5
1,nameOrig,6353307
2,nameDest,2722362


# Feature 상관관계 분석

In [21]:
# 범주형 데이터 확인
categorical_list

['type', 'nameOrig', 'nameDest']

In [22]:
df['isFraud'].value_counts(normalize=True)

0    0.998709
1    0.001291
Name: isFraud, dtype: float64

In [23]:
df['isFraud'].dtype

dtype('int64')

In [25]:
# type 별로 사기건이 몇개 있는지 확인 (CASH_OUT 과 TRANSFER 에서만 사기가 발생함을 알 수 있음)
temp = df.groupby('type')['isFraud'].agg(['count', 'sum'])
temp

,count,sum
type,,
CASH_IN,1399284,0
CASH_OUT,2237500,4116
DEBIT,41432,0
PAYMENT,2151495,0
TRANSFER,532909,4097


In [26]:
# type 별 Target Ratio 확인
temp = df.groupby('type')['isFraud'].agg(['count', 'sum'])
temp['ratio'] = round((temp['sum'] / temp['count'])*100, 2)
temp

,count,sum,ratio
type,,,
CASH_IN,1399284,0,0.00
CASH_OUT,2237500,4116,0.18
DEBIT,41432,0,0.00
PAYMENT,2151495,0,0.00
TRANSFER,532909,4097,0.77
